In [ ]:
library(shiny)

# UI

In [ ]:
ui <- shinyUI(fluidPage(
    titlePanel("Visualize many models"),
    sidebarLayout(
        sidebarPanel(
            h3("Slope"),
            textOutput("slopeOut"),
            h3("Intercept"),
            textOutput("intOut")
        ),
        mainPanel(
            plotOutput("plot1", brush = brushOpts(
                id = "brush1"
            ))
        )
    )
))

# Server

In [ ]:
server <- shinyServer(function(input, output) {
    model <- reactive({
        brushData <- brushedPoints(
            trees,
            input$brush1,
            xvar = "Girth",
            yvar = "Volume"
        )
        if(nrow(brushData) < 2) {
            return(NULL)
        }
        lm(Volume ~ Girth, data = brushData)
    })
    output$slopeOut <- renderText({
        if(is.null(model())) {
            "No model found"
        } else {
            model()[[1]][2]
        }
    })
    output$intOut <- renderText({
        if(is.null(model())) {
            "No model found"
        } else {
            model()[[1]][1]
        }
    })
    output$plot1 <- renderPlot({
        plot(
            trees$Girth,
            trees$Volume,
            xlab = "Girth",
            ylab = "Volume",
            main = "Tree measurements",
            cex = 1.5,
            pch = 16,
            bty = "n"
        )
        if(!is.null(model())) {
            abline(model(), col = "blue", lwd = 2)
        }
    })
})

# Run

In [ ]:
options(shiny.port = 8080)
runApp(shinyApp(ui, server))